In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

INFO 05-19 16:59:25 [__init__.py:239] Automatically detected platform cuda.


In [3]:
model_path = "Qwen/Qwen3-235B-A22B" #"Qwen/Qwen3-235B-A22B" #"Qwen/Qwen3-8B"
tensor_parallel_size = 8 if model_path=="Qwen/Qwen3-235B-A22B" else 1
llm = LLM(model_path, tensor_parallel_size=tensor_parallel_size, max_model_len=32768)
tokenizer = AutoTokenizer.from_pretrained(model_path)

INFO 05-19 16:59:37 [config.py:689] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
INFO 05-19 16:59:38 [config.py:1713] Defaulting to use mp for distributed inference
INFO 05-19 16:59:38 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-19 16:59:39 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='Qwen/Qwen3-235B-A22B', speculative_config=None, tokenizer='Qwen/Qwen3-235B-A22B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observabi

Loading safetensors checkpoint shards:   0% Completed | 0/118 [00:00<?, ?it/s]


(VllmWorker rank=5 pid=834843) INFO 05-19 17:00:05 [weight_utils.py:281] Time spent downloading weights for Qwen/Qwen3-235B-A22B: 0.773510 seconds
(VllmWorker rank=3 pid=834684) INFO 05-19 17:00:06 [weight_utils.py:265] Using model weights format ['*.safetensors']
(VllmWorker rank=6 pid=834916) INFO 05-19 17:00:41 [loader.py:458] Loading weights took 37.18 seconds
(VllmWorker rank=1 pid=834571) INFO 05-19 17:00:41 [loader.py:458] Loading weights took 36.61 seconds
(VllmWorker rank=6 pid=834916) INFO 05-19 17:00:41 [gpu_model_runner.py:1291] Model loading took 54.9205 GiB and 38.637032 seconds
(VllmWorker rank=1 pid=834571) INFO 05-19 17:00:41 [gpu_model_runner.py:1291] Model loading took 54.9205 GiB and 38.779546 seconds
(VllmWorker rank=0 pid=834534) INFO 05-19 17:00:42 [loader.py:458] Loading weights took 37.51 seconds
(VllmWorker rank=0 pid=834534) INFO 05-19 17:00:42 [gpu_model_runner.py:1291] Model loading took 54.9205 GiB and 39.935599 seconds
(VllmWorker rank=4 pid=834768) INFO 

# Zero-shot

In [4]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": "# 社交网络文本本地人身份界定标准多标签分析\n\n## 1. 任务目标\n\n分析给定的社交媒体文本片段，判断**作者本人**在其中明确表达认同、或在字里行间**隐含表达认同**的、用于界定**“本地人”（或其近义词）**的显著标准是什么，并依据本手册RL框架，输出所有相应的RL类别标签。一个文本可能符合**一个或多个**RL类别。\n\n**核心限定:** 本任务**仅关注**作者如何界定“本地人”身份。如果文本仅仅讨论财富、社会地位、生活品质、房产优劣、职业好坏等，而**未明确或强烈暗示**这些是用来判断一个人是否为“本地人”的标准，则**不予标注**。\n\n## 2. 核心原则\n\n*   **聚焦作者核心论点:** 您的判断**必须**基于**发言者（即文本作者）**所展现的、关于“本地人”身份界定的核心观点、主张或评价。理解作者**想说什么**是首要任务。\n*   **分析RL的功能角色:** 识别出的RL元素不仅仅是孤立的特征，更要理解它们在作者构建其核心论点时所扮演的**功能角色**。\n*   **主动推理语境与隐含逻辑:** 需要主动进行语境推理，理解话语背后作者认可的“本地人”身份合法性依据或其评价标准。\n*   **排除抵抗标准与无关讨论:** 对于作者明确表达抵抗、否定，或仅仅描述他人用来针对自己的标准，**不予标注**。对于未与“本地人”身份界定直接关联的讨论，**不予标注**。\n\n## 3. RL (Symbolic Ranking Logic) 框架\n\n**统一句型模板:**\n\"作者认为，人们（或作者本人）应该/常常/可以凭借[RL 类型]所代表的逻辑或标准，来界定谁是/不是‘本地人’，或者评价一个‘本地’区域的好坏/范围。\"\n\n**RL 类型定义与特征:**\n\n*   **RL1：Socially Conventional Territorial Identity（社会习惯性区域认知）**\n    *   **定义:** 作者认同或描述的、基于社会长期积累的习惯性、情感性、历史沉淀的**区域认知、范围界定或人心地图**，这些认知被用来识别“本地人”的身份归属、界定“本地”的地理或心理边界，或进行区域间的社会区隔。\n    *   **核心辨识特征:**\n        *   作者明确提出或默认一种关于**哪些区域“算”或“不算”本地核心区、哪些区域之间存在身份或认知差异**的划分逻辑。\n        *   作者通过描述特定区域的**象征意义、历史标签或社会普遍看法**来暗示其在“本地”身份体系中的地位。\n        *   作者讨论**“市区范围”、“城市边界”等概念的社会认同变迁或当前共识**。\n\n*   **RL2：Administrative Legitimacy（行政归属合法性）**\n    *   **定义:** 作者认同或引用的、基于行政区划、历史并入、法理边界、**户口归属**等法理因素进行的**“本地人”身份归属或区域范围界定**。\n    *   **核心辨识特征:** 关键词包括“划进”、“归属”、“户口是哪的”、“身份证开头”等，作为判断某人/某地在行政法理上是否算“本地”的依据。\n\n*   **RL3：Family Historical Rootedness（家族历史根基/个体成长史）**\n    *   **定义:** 作者认同或引用的、基于**个体或家族在本地的出生、成长历史、代际传承（祖辈、父辈）或早期定居经历**所形成的“本地人”身份根基。在移民城市背景下，可指个体从童年开始的本地化成长史。\n    *   **核心辨识特征:**\n        *   强调“**土生土长**”、“**世代**”、“**祖辈**”、“**父辈**”、“**从小就/幼儿园就来了**”等，以此证明某人是“根正苗红的本地人”或已形成“本地人”的身份认同基础。\n        *   描述因迁移历史（或缺乏迁移史）导致的身份差异。\n\n*   **RL4：Cultural-Linguistic Recognition（文化语言识别性）**\n    *   **定义:** 作者认同或引用的、依据语言、口音、特定地方文化习惯（如习俗、生活方式）来识别某人**是否属于“本地人”群体或具备“本地”特征**。\n    *   **核心辨识特征:** 提及“讲本地话”、“口音”、“懂不懂我们这儿的规矩”等，作为判断内外、判断是否为“自己人”或具备“本地属性”的标准。\n\n*   **RL5：Functional Livability & Environmental Quality（生活功能便利性与环境品质认知）**\n    *   **定义:** 作者认同或引用的、基于地区的生活功能便利性（交通、商业配套、公共服务等）**或**环境品质（如绿化、人少、空气好等宜居因素）进行的**区域优劣认知**，并将这种认知作为评价、选择或定义“理想本地居住区”或界定区域“本地价值”的标准。\n    *   **核心辨识特征:**\n        *   提及“地铁”、“配套”、“方便”、“绿化好”、“人少”等，并**将其与对一个区域是否“好”的评价直接关联**。\n        *   作者因某地的RL5特性而将其视为理想的“本地”生活空间，或因缺乏RL5特性而认为某地不符合“好的本地”标准。\n\n*   **RL6：Social Rootedness & Economic Standing（社会根基深浅与经济地位象征）**\n    *   **定义:** 作者认同或引用的、基于在本地的社会关系网深浅、社区资源掌控（如村社分红）、人脉网络、**拥有本地核心区域特定资产（如房产）或显著经济地位**等，来认知某人**是否算“扎根的本地人”、“有分量的本地人”或其本地身份的稳固性/真实性**。\n    *   **核心辨识特征:**\n        *   提及“人脉”、“分红”、“老关系”、“社区影响力”、“在市中心有几套房”、“房价高/低对身份认同的影响”等，并**明确或隐含地将这些与“本地人身份的稳固性、真实性、层级或对区域的评价标准”挂钩**。\n\n*   **RL7：Occupational Symbolism（职业象征性）**\n    *   **定义:** 作者认同或引用的、基于特定职业身份或区域主要居住人群的职业类型，来区隔**“本地人”与“外来者”**，或判断某职业群体是否代表“传统本地人”或某个“本地”区域的特性。\n    *   **核心辨识特征:** 提及特定职业，并**将其与“本地人”的身份标签或特定区域的“本地属性”强关联**。\n\n## 4. 标注流程\n\n1.  **第一步：理解作者核心表达意图**\n    *   **通读文本，把握作者最想表达的关于“本地人”身份界定（或相关概念）的核心观点、主张、情感或观察是什么？**\n\n2.  **第二步：识别所有潜在RL元素**\n    *   细读文本，找出所有可能与RL框架中各类别相关的具体词语、现象描述、个人经历、因果解释等。初步思考它们可能关联的RL类别。\n\n3.  **第三步：分析RL元素在作者核心论点中的“功能角色”**\n    *   针对每一个识别出的潜在RL元素，判断它在支撑、阐释、反驳、对比或构建作者核心观点时**扮演了什么角色**。\n    *   **判断作者对该元素的态度。**\n\n4.  **第四步：筛选与核心论点相关的、作者认同的RL标准**\n    *   根据上一步的分析，筛选出那些**服务于作者核心论点、且被作者本人明确认同或隐含认同、并清晰用于界定“本地人”身份或评价“本地”区域的RL元素及其逻辑**。\n\n5.  **第五步：精确匹配RL类别并构建解释**\n    *   将筛选出的“合格RL标准/逻辑”与RL框架中的定义进行精确匹配。\n    *   尝试围绕作者的核心观点和关键RL类别，构建一个简洁的解释，说明这些RL是如何共同作用来表达作者意图的。\n\n6.  **第六步：输出结果**\n    *   输出所有匹配到的RL类别标签，用 `, ` 分隔。\n\n## 5. 注意事项\n\n*   时刻提醒自己，标注对象是**作者本人未明确反对的、且用来界定“本地人”身份或评价“本地”区域的标准，并理解其在作者核心论点中的功能**。\n*   不要仅凭关键词进行机械匹配，深入理解作者话语背后的逻辑、意图及其核心观点。**按照 4. 标注流程 中的步骤从第一步一直到第五步地进行思考，不要省略**\n*   在 `输出：`标志后输出结果。输出结果后**立即停止**，不要再继续输出任何内容。"
    },
    {
      "role": "user",
      "content": input_text
    }
  ]

In [5]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 100
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        ) + "第一步：理解作者核心表达意图"
        formatted_prompts_batch.append(formatted_prompt)
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/83 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [6]:
import re
import pandas as pd

# Primary regex for "**输出[:：]**"
primary_output_marker_pattern = re.compile(r"\*\*输出[:：]\*\*\s*(.*)", re.DOTALL)
# Secondary regex for "第六步：输出结果"
# This pattern looks for "第六步：输出结果" followed by optional whitespace,
# and then captures the rest of the string. re.DOTALL ensures `.` matches newlines.
secondary_output_marker_pattern = re.compile(r"第六步：输出结果\s*(.*)", re.DOTALL)
# Regex for splitting by "输出:" or "输出：" - used for the simple/plain marker
simple_output_marker_regex = r"输出[:：]"

outputs = all_outputs # Assuming all_outputs is defined from a previous cell
parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]  # Assuming texts is defined from a previous cell
    model_full_output_text = output_obj.outputs[0].text # The raw text from the model's output

    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary output marker first (bolded)
    match_primary = primary_output_marker_pattern.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (**输出[:：]**)"
    else:
        # If primary marker is not found, try the secondary marker (step-based)
        match_secondary = secondary_output_marker_pattern.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (第六步：输出结果)"
        else:
            # If secondary marker is not found, try the simple plain "输出[:：]" marker.
            # We split by the marker and take the content after the *last* occurrence.
            parts = re.split(simple_output_marker_regex, model_full_output_text)
            if len(parts) > 1: # Marker was found
                extracted_label_part = parts[-1].strip()
                used_marker_type = "simple (plain '输出[:：]' marker)"
    
    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()
        
        # Check if the label is empty after stripping wrappers (e.g., ```\n \n``` or ` `)
        # Also handle cases where the model might output "无" or "（无）" for no labels.
        if not label or label == "无" or label == "（无）":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "无" or "（无）"
                print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label
            
    else:
        # Log a warning if none of the output markers are found
        print(f"Warning: Could not find any of the expected markers: primary ('{primary_output_marker_pattern.pattern}'), secondary ('{secondary_output_marker_pattern.pattern}'), or simple plain ('{simple_output_marker_regex}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename
# Assuming model_path is defined in a previous cell.
# Changed suffix from "_thinking" to "_final_label" to reflect the new parsing focus.
output_csv_filename = f"results/{model_path.split('/')[-1]}_zero_shot.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())


Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Successfully processed 483 model outputs.
Results saved to Qwen3-235B-A22B_zero_shot.csv

First 5 rows of the parsed data:
                                 Original_Input_Text         RL_Types  \
0  热烈欢迎咱叙北第一带盐人 北门可不差，论三环外，东西三环可不一定有北三环好，再等火北弄好，那...        RL1, RL5   
1  真的很多，我就是周边的，不管现实还是网上看的很多，他们天天说自己土著然后说我们弯脚杆，碰到过...        RL1, RL4   
2  我是都江堰的，其他地方的我不敢说，但是我可是从小被成都口音嘲笑哦，我小时候去亲戚家耍过暑假，...  RL4, RL3, RL1   
3  那我说都江堰话被笑的更多，我以前被介绍了个成都人，然后天天给我说他家以前二环以内的，说他家以...               输出：   
4  那证明成都人的确不咋地，本来成都话口音离普通话语区就够偏了，很多怪音，他们还歧视本省的人，无...           输出：RL4   

                                    Raw_Model_Output  
0    \n作者在表达对北门地区的认同和支持，认为北门（北三环）在三环外区域中具有优势，未来随着...  
1    \n作者在表达一种对“本地人”身份认同现象的观察和批评，指出一些自称“土著”的人（尤其是..

# Few-shot

In [7]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": "# **社交网络文本本地人身份界定标准多标签分析**\n\n## 1. 任务目标\n\n分析给定的社交媒体文本片段，判断**作者本人**在其中明确表达认同、或在字里行间**隐含表达认同**的、用于界定**“本地人”（或其近义词，如“土著”、“老街坊”、“是不是我们这里的人”等）**的显著标准是什么，并依据提供的修订版RL框架，输出所有相应的RL类别标签。一个文本可能符合**一个或多个**RL类别。\n\n**核心限定：** 本任务**仅关注**作者如何界定“本地人”身份。如果文本仅仅讨论财富、社会地位、生活品质、房产优劣、职业好坏等，而**未明确或强烈暗示**这些是用来判断一个人是否为“本地人”的标准，则**不予标注**。\n\n## 2. 核心原则\n\n*   **聚焦作者视角:** 您的判断**必须**基于**发言者（即文本作者）**所展现的、**本人认同或隐含认同**的“本地人”身份界定标准和逻辑。\n*   **主动推理语境:** 需要主动进行语境推理，理解话语背后作者认可的“本地人”身份合法性依据。\n*   **排除抵抗标准:** 对于作者明确表达抵抗、否定，或仅仅描述他人用来针对自己的标准，**不予标注**，除非作者在转述或描述时隐含地表达了对该标准的认可或诉诸（即作者本人在某种程度上也认同或利用这个逻辑来界定“本地人”）。\n*   **严格限定于‘本地人’身份标准：** 您的分析和标注**必须**严格围绕作者如何界定“本地人”或与之等同的身份概念。如果文本仅仅讨论一般性的社会分层、财富评价或生活偏好，而**未与“本地人”身份的界定直接关联**，则**不应**进行标注。\n\n## 3. RL (Symbolic Ranking Logic) 框架\n\n**统一句型模板 (辅助理解，聚焦本地人):**\n\"在这个城市/地区，人们（或作者本人）经常用[RL 类型]来判断谁才是真正的‘本地人’，谁不算是‘本地人’\"。\n\n**RL 类型定义与特征 (应用时需紧扣“本地人”界定):**\n\n*   **RL1：Socially Conventional Territorial Identity（社会习惯性区域认知）**\n    *   **定义:** 居民基于习惯性、情感性、历史沉淀的区域认知进行**“本地人”身份识别**与社会区隔。源于社会长期积累的地方话语空间与人心地图。\n    *   **核心辨识特征 (用于界定本地人时):** 强调**“本地人心里”、“我们这儿的人习惯说”、“老一辈讲”**等话语，来界定哪些区域的人是“真本地人”，哪些是“外边的”。呈现为基于区域的“本地/外地”、“真本地/假本地”、“城里人/乡下人”（特指本地语境下的身份区隔）等隐含或显性的身份排序。\n    *   **典型语句 (界定本地人):** “在老成都人心里，出了三环就不是成都了。” “我们这儿的本地人都知道，河东岸的才算老街坊。”\n\n*   **RL2：Administrative Legitimacy（行政归属合法性）**\n    *   **定义:** 人们基于行政区划、历史并入、法理边界、**户口归属**等进行**“本地人”身份的归属认知**。\n    *   **核心辨识特征 (用于界定本地人时):** 关键词包括“划进”、“归属”、“不属于”、“行政区”、“户口是哪的”、“身份证是310开头的”（特指某地）等，用于判断某人是否在行政法理上算“本地人”。\n    *   **典型语句 (界定本地人):** “他户口都不是我们市的，怎么能算本地人？” “我们南汇2009年就合并进浦东新区，我们现在也是名正言顺的浦东本地人了。”\n\n*   **RL3：Family Historical Rootedness（家族历史根基）**\n    *   **定义:** 居民基于**家族**、**血統**、**代際歷史**（祖辈、父辈）在本地的居住传承，及由此衍生的家族层面的“土著”或**“老本地人”身份**。\n    *   **核心辨识特征 (用于界定本地人时):** 关键词**必须**包括或强烈暗示“**世代**”、“**祖辈**”、“**父辈**”、“**家族**”、“**血统**”、“**老家**（指家族几代人居住地）”等，以此证明某人是“根正苗红的本地人”。\n    *   **典型语句 (界定本地人):** “他家**三代**都住这儿，那才是真本地人。” “你**爷爷**那辈儿就来北京了，你当然算老北京。”\n\n*   **RL4：Cultural-Linguistic Recognition（文化语言识别性）**\n    *   **定义:** 居民依据语言、口音、地方文化习惯（如特定习俗、生活方式）来识别某人**是否属于“本地人”群体**。\n    *   **核心辨识特征 (用于界定本地人时):** 关键词包括“讲本地话”、“口音一听就是外地的”、“不懂我们这儿的规矩”等，作为判断内外、判断是否为“自己人”的标准。\n    *   **典型语句 (界定本地人):** “连我们这的方言都不会说，肯定不是本地人。” “看他吃饭的习惯，就知道不是我们这长大的。”\n\n*   **RL5：Functional Livability（生活功能便利性）**\n    *   **定义:** 居民基于地区的生活便利、交通、商业配套等因素进行区域优劣认知，并**可能将其与“真正的本地人居住区”或“外地人聚集区”等身份标签关联**。\n    *   **核心辨识特征 (用于界定本地人时):** 提及“地铁”、“配套”等，**但必须与“本地人”身份的讨论相关联**。例如，作者可能认为“只有住在配套完善的老城区才算真本地人”，或者“新区虽然方便，但住那儿的都感觉不算老市民”。**单纯描述区域便利性而不涉及本地人身份界定，则不适用。**\n    *   **典型语句 (界定本地人，需谨慎判断关联性):** “他们那块儿啥都没有，公交车都少，感觉住的都不是我们市中心的人。” （此例中，功能性被用来区隔“市中心的人”即隐含的“本地核心人群”）\n\n*   **RL6：Social Rootedness（社会根基深浅）**\n    *   **定义:** 居民基于在本地的社会关系网深浅、社区资源掌控（如村社分红）、人脉网络、或**拥有本地核心区域特定资产（如房产）**等，来认知某人**是否算“扎根的本地人”或“有分量的本地人”**。\n    *   **核心辨识特征 (用于界定本地人时):** 提及“人脉”、“分红”、“老关系”、“社区影响力”、“在市中心有几套房”等，并**明确或隐含地将这些与“本地人身份的稳固性或真实性”挂钩**。例如，认为有这些才算“真正在本地扎下根的”。**单纯描述财富或社会资源，若未与“本地人”身份判断关联，则不适用。**\n    *   **典型语句 (界定本地人，需谨慎判断关联性):** “他家在老城有祖产，还有村里股份，这才是牢靠的本地人。” “刚来没几年，连个住的地方都是租的，怎么好意思说自己是本地人？”\n\n*   **RL7：Occupational Symbolism（职业象征性）**\n    *   **定义:** 居民基于区域主要居住人群的职业类型、或特定职业身份，来区隔**“本地人”与“外来者”**，或判断某职业群体是否代表“传统本地人”。\n    *   **核心辨识特征 (用于界定本地人时):** 提及“公务员”、“本地国企员工”、“渔民”、“老手艺人”等职业，并**将其与“本地人”的身份标签或特定区域的“本地属性”强关联**。例如，认为从事某些传统职业的人才是“真本地人”。**单纯描述职业类型或职业聚集区，若未与“本地人”身份判断关联，则不适用。**\n    *   **典型语句 (界定本地人，需谨慎判断关联性):** “我们这片老厂区的子弟，才算得上是真正的本地产业工人后代。” “那些写字楼里的白领好多都是外地来的，不像我们这些从小在机关大院长大的，算是老本地了。”\n\n## 4. 标注流程与详细推理指南\n\n请严格按照以下步骤进行思考和标注：\n\n**步骤一：仔细阅读文本 (Understand the Context)**\n    *   **目标：** 全面理解文本的字面意思、潜在含义以及作者的整体语气和表达意图。\n    *   **行动：** 通读文本至少一遍。注意文本中讨论的核心话题是什么？作者的情绪是怎样的？是否存在明显的对话对象或背景事件？\n\n**步骤二：初步信息提取与梳理 (Extract Key Information)**\n    *   **目标：** 识别并列出文本中可能与身份、地点、评价、标准等相关的关键信息点。此阶段先不进行深层判断，以收集为主。\n    *   **行动：**\n        *   **地理信息：** 记录所有提到的地名、区域描述（如“市区”、“郊区”、“三环内外”）。\n        *   **人群指称：** 记录作者如何称呼自己或他人（如“本地人”、“外地人”、“我们”、“他们”、“老X市人”）。\n        *   **行为/特征：** 记录描述人群或与地点相关的行为、特征、习惯（如“讲XX话”、“住在XX地”、“开车去XX”）。\n        *   **评价/标准：** 记录文本中出现的任何评价性词语或判断标准（如“真正的”、“不算”、“应该”、“才算”）。\n        *   **客观条件/属性：** 记录提及的户籍、房产、职业、居住年限等客观信息。\n\n**步骤三：作者立场分析与“本地人”身份界定关联 (Analyze Author's Stance & Link to \"Local\" Definition)**\n    *   **这是核心判断步骤，务必细致。**\n    *   **目标：** 明确作者本人对于步骤二中提取的各类信息，哪些是TA**认可并用来界定“谁是/不是本地人”或“怎样才算本地人”**的标准或逻辑。\n    *   **行动（按以下子步骤思考）：**\n        *   **3.1 回顾信息：** 逐条回顾步骤二中提取的关键信息点。\n        *   **3.2 筛选标准 - 是否关乎“本地人”界定：**\n            *   **提问自己：** 作者提及这个信息点，是为了说明什么？这个说明最终是指向“如何判断一个人是不是我们这里的人/本地人”吗？\n            *   **正面判断：** 如果信息被作者用来支持或阐述“A是本地人，因为...”或“B不是本地人，因为...”或“要成为本地人，需要...”这类逻辑，则该信息点是重要的候选标准。\n            *   **核心限定应用：** 如果文本仅仅是描述一个地方好不好、房价高不高、生活方不方便，但**没有流露出作者认为这些是判断“本地人”身份的尺子**，则将这些信息点排除。\n        *   **3.3 筛选标准 - 作者是否认可：**\n            *   **提问自己：** 对于这个潜在的“本地人”界定标准，作者本人的态度是什么？是明确赞同？是隐含认同？是客观陈述他人观点但自己不置可否？还是明确反对/批判？\n            *   **排除作者不认可的：**\n                *   如果作者明确说“我不认为...”或“那种说法是错的...”来否定某个标准，则排除。\n                *   如果作者仅仅是转述“有些人用XX标准来排外，我觉得很不好”，这也是排除（除非作者后续流露出自己其实也部分认同该标准的底层逻辑）。\n        *   **3.4 理解构建逻辑：** 对于通过3.2和3.3筛选后留下的、被作者认可且用于界定“本地人”身份的标准：\n            *   **提问自己：** 作者是如何运用这个标准的？是通过它来强调历史渊源？还是地理区域的习惯认知？还是文化习俗的共性？等等。这个标准在作者的论述中起到了什么作用（例如，划清界限、建立优越感、表达包容等）？\n\n**步骤四：匹配RL类别 (Match to RL Categories)**\n    *   **目标：** 将步骤三中识别出的、被作者认可的、且用于界定“本地人”身份的标准或逻辑，与RL框架中的定义进行匹配。\n    *   **行动：**\n        *   **逐个RL核对：** 拿着从步骤三中提炼出的“作者认可的本地人界定标准/逻辑”，逐一对比RL1到RL7的定义和核心辨识特征。\n        *   **寻找最佳匹配：** 哪个RL类型最能概括作者所使用的这个标准/逻辑？\n        *   **关键限定词注意：**\n            *   对于RL5（生活功能便利性）、RL6（社会根基深浅）、RL7（职业象征性），再次确认这些因素**是否被作者明确或强烈地与“本地人身份的界定”联系起来**。如果仅仅是描述现象或个人偏好，则不选。例如，作者说“市中心交通方便”，但没有说“所以住市中心的才是真本地人”，则不选RL5。\n        *   **允许多个标签：** 如果作者同时使用了多种被其认可的标准来界定本地人，则应标注所有匹配的RL类别。例如，作者可能认为“老一辈都说只有住在老城区（RL1）并且家里几代都住这儿（RL3）的才算本地人”。\n\n**步骤五：输出结果 (Output Labels)**\n    *   **目标：** 准确记录所有匹配到的RL类别。\n    *   **行动：**\n        *   输出所有在步骤四中匹配到的RL类别标签，用英文逗号加空格 (`, `) 分隔。\n\n## 5. 示例 (应用详细推理步骤)\n\n**示例输入文本:** \"周边县就是不是哈尔滨呀，我爸老家就是宾县，没有人认为宾县=哈尔滨，在那住的也会区分，而且开车要出城，开高速才能到，都说来哈尔滨和回宾县。至于呼兰阿城双城，更不是一个地方了。小时候坐火车到呼兰要一个小时。我姥姥家行政上属于黑河下面的市，但是从来不会认为这是黑河的一部分，都是各叫各的，我也不认为我去过黑河。只有市区才算是这个市啊。\"\n\n**详细推理过程：**\n\n*   **步骤一：初步信息提取与梳理**\n    *   **地理信息：** 周边县, 哈尔滨, 宾县, 呼兰, 阿城, 双城, 姥姥家(某市), 黑河, 市区。\n    *   **人群指称：** 我, 我爸, 那住的(人), (人们)。\n    *   **行为/特征：** 认为A不等于B, 区分, 开车出城/开高速, 说“来A和回B”, 坐火车, 行政上属于, 不认为A是B的一部分, 各叫各的。\n    *   **评价/标准：** 没有人认为, 更不是一个地方了, 从来不会认为, 只有...才算...。\n    *   **客观条件/属性：** 行政上属于。\n\n*   **步骤二：作者立场分析与“本地人”身份界定关联**\n    *   **2.1 回顾信息：** (同上)\n    *   **2.2 筛选标准 - 是否关乎“本地人”界定：**\n        *   “没有人认为宾县=哈尔滨”、“在那住的也会区分”、“都说来哈尔滨和回宾县” -> 这些社会普遍看法和语言习惯被作者用来论证宾县**不是**哈尔滨（核心部分），间接界定了“哈尔滨人”的范围。**是候选标准。**\n        *   “开车要出城，开高速才能到” -> 物理距离和交通方式被用来支持“不是一个地方”的观点，服务于地理范围的界定。**是候选标准。**\n        *   “呼兰阿城双城，更不是一个地方了” -> 直接判断，界定范围。**是候选标准。**\n        *   “姥姥家行政上属于黑河...但是从来不会认为这是黑河的一部分，都是各叫各的” -> 对比行政归属和实际社会认知的差异，后者被作者强调。**是候选标准（社会认知部分）。**\n        *   “只有市区才算是这个市啊” -> 作者的核心论点，直接界定“市”的范围，即“本地”的范围。**是核心候选标准。**\n    *   **2.3 筛选标准 - 作者是否认可：**\n        *   对于上述所有候选标准（社会普遍看法、语言习惯、物理区隔感、核心市区论），作者都持明确的**认可和赞同**态度，并以此作为自己论证的依据。\n        *   对于“行政归属”（如“姥姥家行政上属于黑河”），作者虽然陈述了事实，但紧接着用“但是从来不会认为...”来表达**不认可**其作为实际地方认同的标准。因此，“行政归属”本身被排除。\n    *   **2.4 理解构建逻辑：**\n        *   作者通过列举一系列社会共识（“没有人认为”、“都说”）、生活经验（“开车出城”、“坐火车”）、以及对“行政区划”与“实际认同”的区分，来构建一个基于**习惯性、约定俗成的、以“市区”为核心的地理区域认知**，这个认知被用来界定“什么是真正的城市（本地）”。\n\n*   **步骤三：匹配RL类别**\n    *   从步骤三提炼出的作者认可的界定逻辑是：一个地方是否属于“本地”，主要看社会大众的普遍认知、日常语言习惯和心理上的区域划分（特别是对“市区”的强调），而不是单纯的行政边界。\n    *   这完全符合 **RL1：Socially Conventional Territorial Identity（社会习惯性区域认知）** 的定义：“居民基于习惯性、情感性、历史沉淀的区域认知进行‘本地人’身份识别与社会区隔。源于社会长期积累的地方话语空间与人心地图。” 文本中的“没有人认为”、“都说”、“各叫各的”、“只有市区才算”都是这种社会习惯性区域认知的体现。\n    *   RL2 (行政归属合法性)：被作者的论述逻辑明确弱化或否定了其作为核心界定标准的作用。\n    *   RL5 (生活功能便利性)：文本中的交通信息（“开车出城”、“坐火车”）更多是用来佐证地理上的区隔感，支持“不是一个地方”的社会认知，而非直接以交通便利性本身来界定“本地人”。核心驱动力是社会习惯认知，而非功能性。\n\n*   **步骤五：输出结果**\n\n**输出:** `RL1`\n\n---\n\n## 6. 注意事项\n\n*   时刻提醒自己，标注对象是**作者本人未明确反对的、且用来界定“本地人”身份的标准**。\n*   不要仅凭关键词进行机械匹配，深入理解作者话语背后的逻辑和意图，**特别是该逻辑是否服务于“本地人”身份的界定**。"
    },
    {
      "role": "user",
      "content": "崇州的嘛，我们身边的人被问到是哪里的，都直接说崇州，不会说是成都的。从小到大耳濡目染的就是成都人都觉得三环以外的不算成都的。"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：\n作者想表达的是自己以及身边人在认同“本地人”身份时，会直接说“崇州”，而不会说“成都”；并且，作者从小受到的社会习惯性认知是，成都人普遍认为城三环以外的区域不算是“成都”本地核心区。这反映了一种基于地理范围和社会习俗界定“本地人”的标准。\n\n2. 识别所有潜在RL元素：\n- 身边人认同自己的区域是“崇州”，不说“成都”，体现的是一种区域认知和身份界定。\n- “从小到大耳濡目染”说明这种认知是长期社会习惯性的。\n- “三环以外的不算成都的”明确划分了地理范围，界定哪些区域被视为本地核心区域。\n\n3. 分析RL元素在作者核心论点中的功能角色：\n这些地理区域的界定，连同作者和周围人的认同习惯，共同构成了一套社会习惯性的区域认知标准，用以界定谁是本地人，从而确认“崇州”与“成都”的本地人身份差异。\n\n4. 筛选与核心论点相关、作者认同的RL标准：\n作者认同并隐含表达了基于长期形成的社会习惯性区域认知（如三环划分及所属城市认同）来界定“本地人”身份。\n\n5. 精确匹配RL类别并构建解释：\n该文本符合RL1，因为作者强调通过社会长期习惯认知和区域范围划分的方式，来判断和表述“本地人”的归属和身份。\n\n输出：RL1"
    },
    {
      "role": "user",
      "content": "抱抱，反正在我眼里我只看行政区域划分，属于青岛那就是青岛人，管是胶州还是城阳，早些年因为交通不便利人们觉得隔着远就罢了，现在不管地铁还是开车一会就到了呀"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：  \n作者明确表示，他认同“本地人”的身份归属应该依据行政区域划分，即只要属于青岛行政区域（包含胶州、城阳），就被认定为青岛人，并以此作为判断本地人身份的标准。同时提及交通便利性改善，改变了以往因距离远而不被认同的观念。\n\n2. 识别所有潜在RL元素：  \n- “只看行政区域划分”、“属于青岛那就是青岛人”，突出了行政归属作为界定标准。  \n- 提到了具体区划（胶州、城阳），表明行政区划内成员被视作本地人。  \n- “交通不便利”、“地铁”、“开车一会就到了”，虽有生活功能便利性描述，但作为交通便利促进的背景说明，强调区划重要性的变化。\n\n3. 分析RL元素在作者核心论点中的功能角色：  \n行政区划是作者用来界定“本地人”身份的根本标准，交通便利性作为补充说明推动对本地身份认知的变化，但不是本地人身份的标准。\n\n4. 筛选与核心论点相关、作者认同的RL标准：  \n作者显著认同以行政区划作为判断“本地人”的标准。\n\n5. 精确匹配RL类别并构建解释：  \n本段文本反映了基于行政区划合法性和归属的本地人身份界定，符合RL2。\n\n输出：RL2"
    },
    {
      "role": "user",
      "content": "在广州三代以内上就算真正广州人，二代以内是新广州人，一代开荒牛是广州移民，三代都不讲广州话的是外省广州人（祖宗是捞佬）来广州的。"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：  \n作者明确表达了以家族世代（代际传承）作为界定“本地人”身份的标准，并通过不同代际划分“真正广州人”、“新广州人”、“广州移民”以及“外省广州人”等身份层级，强调家族根基的重要性。\n\n2. 识别所有潜在RL元素：  \n- “三代以内”、“二代以内”、“一代开荒牛”明显是基于家族历史根基和成长史的界定。  \n- “三代都不讲广州话”暗含文化语言的识别。  \n- 但核心论点的主轴还是以家族世代为划分线，强调血缘、传承和代际的认同。\n\n3. 分析RL元素在作者核心论点中的功能角色：  \n家族历史根基是作者用来严格界定广州本地人身份的主要标准，世代传承决定了“真正的”身份归属。文化语言虽有提及，但更多用于描述特定代际的文化差异，而非作为主要标准。\n\n4. 筛选与核心论点相关、作者认同的RL标准：  \n作者认同并明确表达家族历史根基（家族世代传承）作为本地人身份界定的核心标准。\n\n5. 精确匹配RL类别并构建解释：  \n该文本主要体现基于家族历史根基的本地人身份认定逻辑，符合RL3。\n\n输出：RL3"
    },
    {
      "role": "user",
      "content": "我是成都土著，我觉得还好！说实话哈，只是确确实实有一点受不了一些口音。"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：  \n作者自称“成都土著”，表明其本地人身份；同时提及“受不了一些口音”，暗示语言口音是识别“本地人”与“非本地人”的一个标准或边界。\n\n2. 识别所有潜在RL元素：  \n- “我是成都土著”表明家乡出生或长期生活的身份认同。  \n- “受不了一些口音”涉及语言文化的辨识。  \n- 口音作为本地人与非本地人的区别标志。\n\n3. 分析RL元素在作者核心论点中的功能角色：  \n作者通过自我认同的“土著”身份和对口音的敏感体现了文化语言识别作为判断本地人与非本地人的标准，暗含对文化语言的认同或排斥。\n\n4. 筛选与核心论点相关、作者认同的RL标准：  \n作者隐含认同口音等文化语言差异作为界定本地人身份的标准。\n\n5. 精确匹配RL类别并构建解释：  \n该文本符合基于语言与文化特点来识别本地人身份的RL4类别。\n\n输出：RL4"
    },
    {
      "role": "user",
      "content": "所以我喜欢北辰，老城区往新区的过渡，街道界面没老城区那么破旧，离市中心又只需要十几分钟，配套啥都有，住起来太舒服哒"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：  \n作者表达了对北辰这个区域的喜爱，主要基于该区域的生活便利性和环境品质，如街道状况、离市中心的距离和配套设施，认为这里“住起来太舒服”。这体现了作者对本地不同区域生活环境优劣的评价标准。\n\n2. 识别所有潜在RL元素：  \n- “街道界面没老城区那么破旧”体现环境品质差异。  \n- “离市中心只需要十几分钟”，强调交通便利。  \n- “配套啥都有”，指生活功能便利。  \n- “住起来舒服”，反映区域生活品质的总体评价。\n\n3. 分析RL元素在作者核心论点中的功能角色：  \n这些描述支撑作者对北辰作为一个本地区域的高生活品质评价，暗含该区域较理想的本地居住条件，体现了作者用生活便利和环境品质作为评价本地区域优劣及“本地价值”的标准。\n\n4. 筛选与核心论点相关、作者认同的RL标准：  \n作者明确将生活环境和配套便利性作为衡量和选择理想本地居住区的标准。\n\n5. 精确匹配RL类别并构建解释：  \n文本符合基于生活功能便利性与环境品质认知的RL5标准，即以生活舒适度及配套设施优劣来评价本地区域的等级和“本地价值”。\n\n输出：RL5"
    },
    {
      "role": "user",
      "content": "毕竟好多家庭不见得可以拿出一两百万买一个环境好点、还在二环内的房子。毕竟我爸妈在成都工作这么久，基本都在一二环内，朋友也在附近，买到郊区肯定不现实，买一二环的房子改造是最好的方案了。"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：  \n作者表达了经济能力限制和社会关系网络对选购房产区域的影响，认为拥有二环内房产且靠近工作和朋友的区域，是更现实且理想的生活选择，暗含房产位置和经济条件作为“本地人”身份的体现和区域优劣的评价标准。\n\n2. 识别所有潜在RL元素：  \n- “买一个环境好点、还在二环内的房子”强调房产地理位置和品质。  \n- “爸妈在成都工作这么久”、“朋友也在附近”体现社会关系和生活根基。  \n- “买到郊区肯定不现实”，表明对“本地好区”与“边缘区”的区分。  \n- “买一二环的房子改造是最好的方案”，体现经济条件与生活环境的综合考量。\n\n3. 分析RL元素在作者核心论点中的功能角色：  \n作者隐含将经济实力、已有社会关系（亲戚朋友分布）、以及拥有核心区域房产作为判断本地人身份稳固性和区域价值的重要标准，表达对二环内房产及其社会根基的认同。\n\n4. 筛选与核心论点相关、作者认同的RL标准：  \n作者隐含认可拥有核心区（如二环内）的房产及稳定社会关系，作为本地人身份和区域优劣的重要标准。\n\n5. 精确匹配RL类别并构建解释：  \n本段文本主要体现社会根基与经济地位象征对本地人身份认定的影响，符合RL6。\n\n输出：RL6"
    },
    {
      "role": "user",
      "content": "李沧黄岛那里是农村，很多种地的农民，老一辈不认为他们是城里人。"
    },
    {
      "role": "assistant",
      "content": "1. 理解作者核心表达意图：  \n作者提到“农民”作为一种职业身份，并暗示老一辈对于“城里人”的认同排斥种地的农民，说明职业身份在本地人界定中承担一定作用。\n\n2. 识别所有潜在RL元素：  \n- “农民”作为具体职业身份。  \n- “老一辈不认为他们是城里人”，暗示基于职业的身份排斥或界定。\n\n3. 分析RL元素在作者核心论点中的功能角色：  \n职业身份实际上被用来区分“城里人”与非城里人的界定标准，具有象征意义。\n\n4. 筛选与核心论点相关、作者认同的RL标准：  \n作者认可职业身份（农民）作为区分本地人与非本地人或不同社会群体的界定标准。\n\n5. 精确匹配RL类别并构建解释：  \n“农民”作为职业身份用来界定本地人身份的做法符合职业象征性标准，判定为RL7。\n\n输出：RL7"
    },
    {
      "role": "user",
      "content": input_text
    },
  ]

In [8]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 100
# sampling_params = SamplingParams(temperature=0.6, top_p=0.95, top_k=20, min_p=0, max_tokens=8192)
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        ) + "第一步：理解作者核心表达意图"

        formatted_prompts_batch.append(formatted_prompt)
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/83 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [9]:
import re
import pandas as pd

# Primary regex for "**输出[:：]**"
primary_output_marker_pattern = re.compile(r"\*\*输出[:：]\*\*\s*(.*)", re.DOTALL)
# Secondary regex for "第六步：输出结果"
# This pattern looks for "第六步：输出结果" followed by optional whitespace,
# and then captures the rest of the string. re.DOTALL ensures `.` matches newlines.
secondary_output_marker_pattern = re.compile(r"第六步：输出结果\s*(.*)", re.DOTALL)
# Regex for splitting by "输出:" or "输出：" - used for the simple/plain marker
simple_output_marker_regex = r"输出[:：]"

outputs = all_outputs # Assuming all_outputs is defined from a previous cell
parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]  # Assuming texts is defined from a previous cell
    model_full_output_text = output_obj.outputs[0].text # The raw text from the model's output

    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary output marker first (bolded)
    match_primary = primary_output_marker_pattern.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (**输出[:：]**)"
    else:
        # If primary marker is not found, try the secondary marker (step-based)
        match_secondary = secondary_output_marker_pattern.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (第六步：输出结果)"
        else:
            # If secondary marker is not found, try the simple plain "输出[:：]" marker.
            # We split by the marker and take the content after the *last* occurrence.
            parts = re.split(simple_output_marker_regex, model_full_output_text)
            if len(parts) > 1: # Marker was found
                extracted_label_part = parts[-1].strip()
                used_marker_type = "simple (plain '输出[:：]' marker)"
    
    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()
        
        # Check if the label is empty after stripping wrappers (e.g., ```\n \n``` or ` `)
        # Also handle cases where the model might output "无" or "（无）" for no labels.
        if not label or label == "无" or label == "（无）":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "无" or "（无）"
                print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label
            
    else:
        # Log a warning if none of the output markers are found
        print(f"Warning: Could not find any of the expected markers: primary ('{primary_output_marker_pattern.pattern}'), secondary ('{secondary_output_marker_pattern.pattern}'), or simple plain ('{simple_output_marker_regex}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename
# Assuming model_path is defined in a previous cell.
# Changed suffix from "_thinking" to "_final_label" to reflect the new parsing focus.
output_csv_filename = f"results/{model_path.split('/')[-1]}_few_shot.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())

Successfully processed 483 model outputs.
Results saved to Qwen3-235B-A22B_few_shot.csv

First 5 rows of the parsed data:
                                 Original_Input_Text           RL_Types  \
0  热烈欢迎咱叙北第一带盐人 北门可不差，论三环外，东西三环可不一定有北三环好，再等火北弄好，那...                RL1   
1  真的很多，我就是周边的，不管现实还是网上看的很多，他们天天说自己土著然后说我们弯脚杆，碰到过...          RL1, RL4   
2  我是都江堰的，其他地方的我不敢说，但是我可是从小被成都口音嘲笑哦，我小时候去亲戚家耍过暑假，...                RL4   
3  那我说都江堰话被笑的更多，我以前被介绍了个成都人，然后天天给我说他家以前二环以内的，说他家以...  RL1, RL3, RL4**   
4  那证明成都人的确不咋地，本来成都话口音离普通话语区就够偏了，很多怪音，他们还歧视本省的人，无...               （无标签）   

                                    Raw_Model_Output  
0    \n作者以“叙北第一带盐人”“北门可不差”等表达对北门区域的自豪感和认同感，并通过比较“...  
1    \n作者通过自身经历和观察，表达了“自称土著”的群体如何以**居住区域（市中心 vs 周...  
2    \n作者强调自己是“都江堰的”，并详细描述了因**口音差异**而遭受成都本地人嘲笑的经历...  
3    \n作者描述了自己使用都江堰方言时被嘲笑的经历，并提到一个“成都人”反复强调自己家族住在...  
4    \n作者对成都本地人（尤其是说成都话的人）持批评态度，指出成都话口音偏、怪，并批评成都人...  


In [10]:
results_df['RL_Types']

0                    RL1
1              RL1, RL4
2                    RL4
3      RL1, RL3, RL4**
4                   （无标签）
              ...        
478                RL1**
479                 （无标签）
480                  RL1
481                  RL2
482                  RL1
Name: RL_Types, Length: 483, dtype: object

# No CoT

In [11]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": "# 社交网络文本本地人身份界定标准多标签分析\n\n## 1. 任务目标\n\n分析给定的社交媒体文本片段，判断**作者本人**在其中明确表达认同、或在字里行间**隐含表达认同**的、用于界定**“本地人”（或其近义词）**的显著标准是什么，并依据本手册RL框架，输出所有相应的RL类别标签。一个文本可能符合**一个或多个**RL类别。\n\n**核心限定:** 本任务**仅关注**作者如何界定“本地人”身份。如果文本仅仅讨论财富、社会地位、生活品质、房产优劣、职业好坏等，而**未明确或强烈暗示**这些是用来判断一个人是否为“本地人”的标准，则**不予标注**。\n\n## 2. 核心原则\n\n*   **聚焦作者核心论点:** 您的判断**必须**基于**发言者（即文本作者）**所展现的、关于“本地人”身份界定的核心观点、主张或评价。理解作者**想说什么**是首要任务。\n*   **分析RL的功能角色:** 识别出的RL元素不仅仅是孤立的特征，更要理解它们在作者构建其核心论点时所扮演的**功能角色**。\n*   **主动推理语境与隐含逻辑:** 需要主动进行语境推理，理解话语背后作者认可的“本地人”身份合法性依据或其评价标准。\n*   **排除抵抗标准与无关讨论:** 对于作者明确表达抵抗、否定，或仅仅描述他人用来针对自己的标准，**不予标注**。对于未与“本地人”身份界定直接关联的讨论，**不予标注**。\n\n## 3. RL (Symbolic Ranking Logic) 框架\n\n**统一句型模板:**\n\"作者认为，人们（或作者本人）应该/常常/可以凭借[RL 类型]所代表的逻辑或标准，来界定谁是/不是‘本地人’，或者评价一个‘本地’区域的好坏/范围。\"\n\n**RL 类型定义与特征:**\n\n*   **RL1：Socially Conventional Territorial Identity（社会习惯性区域认知）**\n    *   **定义:** 作者认同或描述的、基于社会长期积累的习惯性、情感性、历史沉淀的**区域认知、范围界定或人心地图**，这些认知被用来识别“本地人”的身份归属、界定“本地”的地理或心理边界，或进行区域间的社会区隔。\n    *   **核心辨识特征:**\n        *   作者明确提出或默认一种关于**哪些区域“算”或“不算”本地核心区、哪些区域之间存在身份或认知差异**的划分逻辑。\n        *   作者通过描述特定区域的**象征意义、历史标签或社会普遍看法**来暗示其在“本地”身份体系中的地位。\n        *   作者讨论**“市区范围”、“城市边界”等概念的社会认同变迁或当前共识**。\n\n*   **RL2：Administrative Legitimacy（行政归属合法性）**\n    *   **定义:** 作者认同或引用的、基于行政区划、历史并入、法理边界、**户口归属**等法理因素进行的**“本地人”身份归属或区域范围界定**。\n    *   **核心辨识特征:** 关键词包括“划进”、“归属”、“户口是哪的”、“身份证开头”等，作为判断某人/某地在行政法理上是否算“本地”的依据。\n\n*   **RL3：Family Historical Rootedness（家族历史根基/个体成长史）**\n    *   **定义:** 作者认同或引用的、基于**个体或家族在本地的出生、成长历史、代际传承（祖辈、父辈）或早期定居经历**所形成的“本地人”身份根基。在移民城市背景下，可指个体从童年开始的本地化成长史。\n    *   **核心辨识特征:**\n        *   强调“**土生土长**”、“**世代**”、“**祖辈**”、“**父辈**”、“**从小就/幼儿园就来了**”等，以此证明某人是“根正苗红的本地人”或已形成“本地人”的身份认同基础。\n        *   描述因迁移历史（或缺乏迁移史）导致的身份差异。\n\n*   **RL4：Cultural-Linguistic Recognition（文化语言识别性）**\n    *   **定义:** 作者认同或引用的、依据语言、口音、特定地方文化习惯（如习俗、生活方式）来识别某人**是否属于“本地人”群体或具备“本地”特征**。\n    *   **核心辨识特征:** 提及“讲本地话”、“口音”、“懂不懂我们这儿的规矩”等，作为判断内外、判断是否为“自己人”或具备“本地属性”的标准。\n\n*   **RL5：Functional Livability & Environmental Quality（生活功能便利性与环境品质认知）**\n    *   **定义:** 作者认同或引用的、基于地区的生活功能便利性（交通、商业配套、公共服务等）**或**环境品质（如绿化、人少、空气好等宜居因素）进行的**区域优劣认知**，并将这种认知作为评价、选择或定义“理想本地居住区”或界定区域“本地价值”的标准。\n    *   **核心辨识特征:**\n        *   提及“地铁”、“配套”、“方便”、“绿化好”、“人少”等，并**将其与对一个区域是否“好”的评价直接关联**。\n        *   作者因某地的RL5特性而将其视为理想的“本地”生活空间，或因缺乏RL5特性而认为某地不符合“好的本地”标准。\n\n*   **RL6：Social Rootedness & Economic Standing（社会根基深浅与经济地位象征）**\n    *   **定义:** 作者认同或引用的、基于在本地的社会关系网深浅、社区资源掌控（如村社分红）、人脉网络、**拥有本地核心区域特定资产（如房产）或显著经济地位**等，来认知某人**是否算“扎根的本地人”、“有分量的本地人”或其本地身份的稳固性/真实性**。\n    *   **核心辨识特征:**\n        *   提及“人脉”、“分红”、“老关系”、“社区影响力”、“在市中心有几套房”、“房价高/低对身份认同的影响”等，并**明确或隐含地将这些与“本地人身份的稳固性、真实性、层级或对区域的评价标准”挂钩**。\n\n*   **RL7：Occupational Symbolism（职业象征性）**\n    *   **定义:** 作者认同或引用的、基于特定职业身份或区域主要居住人群的职业类型，来区隔**“本地人”与“外来者”**，或判断某职业群体是否代表“传统本地人”或某个“本地”区域的特性。\n    *   **核心辨识特征:** 提及特定职业，并**将其与“本地人”的身份标签或特定区域的“本地属性”强关联**。\n\n## 4. 注意事项\n\n*   时刻提醒自己，标注对象是**作者本人未明确反对的、且用来界定“本地人”身份或评价“本地”区域的标准，并理解其在作者核心论点中的功能**。\n*   不要仅凭关键词进行机械匹配，深入理解作者话语背后的逻辑、意图及其核心观点。\n*   在 `输出：`标志后输出结果。输出结果后**立即停止**，不要再继续输出任何内容。"
    },
    {
      "role": "user",
      "content": input_text
    }
  ]

In [12]:
all_outputs = []
batch_size = 100
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        )
        formatted_prompts_batch.append(formatted_prompt + "输出：")
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/83 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [13]:
import re
import pandas as pd
# Regex to find the start of the final output section.
# It looks for "输出:" or "输出：" (with either half-width or full-width colon),
# followed by optional whitespace, and then captures the rest of the string (which is the model's raw output).
output_marker_pattern = re.compile(r"输出[:：]\s*(.*)", re.DOTALL) # Removed \*\* to be more flexible
outputs = all_outputs
parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]  # The original text fed into the prompt
    # Construct the string to be parsed. The model's output (output_obj.outputs[0].text)
    # is what comes after the "输出：" prompt.
    model_full_output_text = "输出： " + output_obj.outputs[0].text

    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    match = output_marker_pattern.search(model_full_output_text)
    if match:
        # Extract the content after the "输出[:：]\s*" marker
        # This is essentially output_obj.outputs[0].text, but with leading/trailing whitespace potentially handled by \s*
        extracted_label_part = match.group(1).strip()
        
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: Potentially **LABEL** or just LABEL
        # If it's **LABEL**, this will keep the asterisks. This might be desired or handled later.
        # The main goal here is to capture the label content.
        else:
            label = extracted_label_part.strip() # .strip() handles if extracted_label_part was just spaces
        
        # Check if the label is empty after stripping wrappers (e.g., ```\n \n``` or ` ` or if model output was empty)
        if not label:
            current_parsed_item["RL_Types"] = "PARSE_ERROR_EMPTY_LABEL"
            print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i}.")
            print(f"Problematic Raw Output (after marker search, before stripping):\n'{match.group(1)}'\n" + "="*30)
            print(f"Original model output part (output_obj.outputs[0].text):\n'{output_obj.outputs[0].text}'\n" + "="*30)

        else:
            current_parsed_item["RL_Types"] = label
            
    else:
        # Log a warning if the output marker is not found
        # The marker string in the warning is a human-readable representation of what was expected.
        print(f"Warning: Could not find the output marker pattern '输出[:：]\\s*(.*)' in model output for original text at index {i}.")
        print(f"Problematic Raw Output (model_full_output_text):\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename
# Assuming model_path is defined in a previous cell.
# Changed suffix from "_thinking" to "_final_label" to reflect the new parsing focus.
output_csv_filename = f"results/{model_path.split('/')[-1]}_no_cot.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())


Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outputs[0].text):
''
Problematic Raw Output (after marker search, before stripping):
''
Original model output part (output_obj.outpu

In [14]:
results_df['RL_Types']

0                                       RL1
1                        RL1 RL3 RL4 RL2
2                                 RL3, RL4
3      RL4（Cultural-Linguistic Recognition）
4                                       RL4
                       ...                  
478                               RL1, RL3
479                                     RL3
480                                     RL4
481                               RL2, RL1
482                                     RL1
Name: RL_Types, Length: 483, dtype: object